In [1]:
%%capture
import datajoint_plus as djp
djp.config['database.user'] = 'micronsdashboard'
djp.config['database.password'] = 'micronsdashboard#1'

from IPython.core.magics.display import Javascript
import wridgets as wr
import wridgets.app as wra
from microns_dashboard_api.apps import GetDashboardUser, UserApp, DataJointLoginApp, UserInfoManager
from microns_dashboard_api.schemas import dashboard as db

In [3]:
Javascript(UserApp.get_user_info_js) # must be in its own cell

<IPython.core.display.Javascript object>

In [4]:
user = GetDashboardUser() # must be in its own cell
user

GetDashboardUser()

In [2]:
# user app
header_output = wr.Output()
display(header_output)
def on_user_update():
    event = db.Event.log_event('user_access', {'user': user.name})
    with header_output:
        wra.Label(text=f'Welcome, {user.name}.', fontsize=2.5).display()
        
user_app = UserApp(user_app=globals().get('user'), on_user_update=on_user_update, entry_point='UserManager', post_event=True)

Output()

In [ ]:
# Slack username 
class SlackUsernameManager(UserInfoManager):
    def get_data(self, **kwargs):
        try:
            return db.User.Slack().get({'user': user_app.user}).get('slack_username')
        except:
            msg = 'Error getting Slack username'
            db.User.Slack.Log('exception', msg)
            self.msg(msg)

    def set_data(self, data, **kwargs):
        if data:
            event = None
            with wr.Output():
                event = db.Event.log_event('user_add_info', {'user': user_app.user, 'info_type': 'slack_username'}, data)
            if event is not None:
                self.msg(f'Slack username successfully updated to <b> {data} </b>')
            else:
                self.msg(f'Error updating Slack username to <b> {data} </b>')

In [4]:
# DataJoint login then display apps

app_output = wr.Output()

def on_login(**kwargs):
    with app_output:
        SlackUsernameManager(label='Slack username').display()
        
DataJointLoginApp(on_login=on_login, hide_on_login=True).display()
display(app_output)